In [1]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

from data import get_dataloader
from lightning import ViT

Global seed set to 42


Device: cuda:0


Global seed set to 42


Device: cuda:0


In [2]:
data_dir = './big'
EPOCHS = 5
bs = 8
LEARNING_RATE = 2e-5
pretrained_model_path = "google/vit-base-patch16-224-in21k"

In [3]:
train_loader, val_loader, test_loader, num_class = get_dataloader(
    dataset_folder=data_dir, 
    BATCH_SIZE=bs, 
    val_split=0.20, 
    pretrained_model_path=pretrained_model_path)

Train dataset : 147
Validation dataset : 37
Test dataset : 47


In [4]:
model_kwargs = {
    "num_class" : num_class,
    "pretrained_model_path" : pretrained_model_path
}

In [5]:
model = ViT(model_kwargs, lr=LEARNING_RATE)

In [6]:
trainer = pl.Trainer(
    max_epochs=EPOCHS, 
    gpus=1 if str(device) == "cuda:0" else 0,
    default_root_dir=".",
    log_every_n_steps=5,
    callbacks=[ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc")]
    )
trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                      | Params
----------------------------------------------------
0 | model | ViTForImageClassification | 86.4 M
----------------------------------------------------
86.4 M    Trainable params
0         Non-trainable params
86.4 M    Total params
345.563   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [7]:
test_result = trainer.test(model, dataloaders=test_loader, verbose=False)
test_result[0]["test_acc"]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

0.8936170339584351